In [1]:
import dask.dataframe as dd
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

In [2]:
# load crimes parquet data into dask df
data_file_name = '../data/crimes-2017-7-4.parq'
crimes = dd.read_parquet(data_file_name,
                        index='Date')

In [3]:
%%time
# log records count and load data partitions
print("{:,} total records in {} partitions".format(len(crimes), crimes.npartitions))
print("DataFrame size: {:,}".format(crimes.size.compute()))

129,956 total records in 1 partitions
DataFrame size: 1,039,648
Wall time: 2.59 s


In [4]:
# check data frame structure
crimes

,Block,PrimaryType,Description,LocationDescription,Arrest,Domestic,Latitude,Longitude
npartitions=1,,,,,,,,
2017-01-01 00:00:00,object,object,object,object,bool,bool,float64,float64
2017-07-04 23:50:00,...,...,...,...,...,...,...,...


In [5]:
# preview crimes data with Date index
crimes.head()

,Block,PrimaryType,Description,LocationDescription,Arrest,Domestic,Latitude,Longitude
Date,,,,,,,,
2017-07-04 18:39:00,002XX E 132ND ST,HOMICIDE,FIRST DEGREE MURDER,AUTO,False,False,41.655619,-87.616339
2017-04-11 14:00:00,038XX N LINCOLN AVE,THEFT,OVER $500,RESTAURANT,False,False,NaN,NaN
2017-05-20 14:00:00,008XX N HAMLIN AVE,OTHER OFFENSE,TELEPHONE THREAT,APARTMENT,False,False,NaN,NaN
2017-07-04 08:00:00,043XX N CICERO AVE,DECEPTIVE PRACTICE,"THEFT BY LESSEE,MOTOR VEH",OTHER,False,False,NaN,NaN
2017-07-04 07:14:00,014XX W 79TH ST,CRIMINAL DAMAGE,TO PROPERTY,STREET,False,False,NaN,NaN


In [6]:
# get crime types
crime_types = crimes[['PrimaryType']]

In [7]:
crime_types.head()

,PrimaryType
Date,
2017-07-04 18:39:00,HOMICIDE
2017-04-11 14:00:00,THEFT
2017-05-20 14:00:00,OTHER OFFENSE
2017-07-04 08:00:00,DECEPTIVE PRACTICE
2017-07-04 07:14:00,CRIMINAL DAMAGE


In [8]:
# get crime counts by primary type
crime_type_total = crime_types.groupby('PrimaryType').size().compute()
print(crime_type_total)

PrimaryType
ARSON                                  233
ASSAULT                               9695
BATTERY                              24708
BURGLARY                              6361
CONCEALED CARRY LICENSE VIOLATION       29
CRIM SEXUAL ASSAULT                    716
CRIMINAL DAMAGE                      14525
CRIMINAL TRESPASS                     3518
DECEPTIVE PRACTICE                    8360
GAMBLING                                90
HOMICIDE                               329
HUMAN TRAFFICKING                        5
INTERFERENCE WITH PUBLIC OFFICER       568
INTIMIDATION                            71
KIDNAPPING                              93
LIQUOR LAW VIOLATION                   109
MOTOR VEHICLE THEFT                   5669
NARCOTICS                             5198
NON-CRIMINAL                            18
NON-CRIMINAL (SUBJECT SPECIFIED)         2
OBSCENITY                               33
OFFENSE INVOLVING CHILDREN            1085
OTHER NARCOTIC VIOLATION                 1

In [9]:
# get arrests and domestic counts for each primary crime type
crime_type_counts = crimes[['PrimaryType', 'Arrest', 'Domestic']].groupby('PrimaryType').sum().compute()
print(crime_type_counts)

                                   Arrest  Domestic
PrimaryType                                        
ARSON                                14.0       8.0
ASSAULT                            1700.0    2493.0
BATTERY                            4945.0   12020.0
BURGLARY                            219.0      86.0
CONCEALED CARRY LICENSE VIOLATION    28.0       0.0
CRIM SEXUAL ASSAULT                  24.0     108.0
CRIMINAL DAMAGE                     857.0    1682.0
CRIMINAL TRESPASS                  2163.0     161.0
DECEPTIVE PRACTICE                  307.0      77.0
GAMBLING                             90.0       0.0
HOMICIDE                             47.0      16.0
HUMAN TRAFFICKING                     0.0       2.0
INTERFERENCE WITH PUBLIC OFFICER    538.0       3.0
INTIMIDATION                          4.0       6.0
KIDNAPPING                            8.0      31.0
LIQUOR LAW VIOLATION                109.0       1.0
MOTOR VEHICLE THEFT                 461.0      51.0
NARCOTICS   

In [10]:
crime_type_counts['Total'] = crime_type_total

In [11]:
print(crime_type_counts)

                                   Arrest  Domestic  Total
PrimaryType                                               
ARSON                                14.0       8.0    233
ASSAULT                            1700.0    2493.0   9695
BATTERY                            4945.0   12020.0  24708
BURGLARY                            219.0      86.0   6361
CONCEALED CARRY LICENSE VIOLATION    28.0       0.0     29
CRIM SEXUAL ASSAULT                  24.0     108.0    716
CRIMINAL DAMAGE                     857.0    1682.0  14525
CRIMINAL TRESPASS                  2163.0     161.0   3518
DECEPTIVE PRACTICE                  307.0      77.0   8360
GAMBLING                             90.0       0.0     90
HOMICIDE                             47.0      16.0    329
HUMAN TRAFFICKING                     0.0       2.0      5
INTERFERENCE WITH PUBLIC OFFICER    538.0       3.0    568
INTIMIDATION                          4.0       6.0     71
KIDNAPPING                            8.0      31.0     